In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
data_path = "/content/drive/MyDrive/Capstone/Table/"

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.corpus import stopwords
from wordcloud import WordCloud
from nltk import word_tokenize, sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer 
import string
import re
import nltk

import spacy

In [4]:
df = pd.read_csv(data_path+"LDA3_emails_assignments.csv")
df1 = pd.read_csv(data_path+"data_splited_3.csv")

## Sample

In [ ]:
df1

,Unnamed: 0,Unnamed: 0.1,email_id,subject,email,clean_body,clean_subject
0,0,0,allen-p\all_documents\1#1,"December 14, 2000 - Bear Stearns' predictions ...",In today's Daily Update you'll find free repor...,today daily update find free report america on...,december 2000 bear stearns prediction telecom ...
1,1,1,allen-p\all_documents\10#1,Bloomberg Power Lines Report,Here is today's copy of Bloomberg Power Lines....,today copy bloomberg power lines adobe acrobat...,bloomberg power lines report
2,2,2,allen-p\all_documents\100#1,Consolidated positions: Issues & To Do list,From our initial set of meetings with the trad...,initial set meeting trader regarding consolida...,consolidated position issues list
3,4,4,allen-p\all_documents\100#3,Consolidated positions: Issues & To Do list,,NaN,consolidated position issues list
4,11,11,allen-p\all_documents\104#2,"Var, Reporting and Resources Meeting",,NaN,var reporting resources meeting
...,...,...,...,...,...,...,...
364181,123313,123313,zufferli-j\sent_items\85#1,Generation Maintenance Planning and Related Is...,"\t\t> You, or a representative...",representative organization invited attend mee...,generation maintenance planning related issues...
364182,123314,123314,zufferli-j\sent_items\86#3,RE: hello,going great. Even the weather has been record...,going great even weather record warm think hap...,hello
364183,123317,123317,zufferli-j\sent_items\93#1,Hello,"Good to hear your voice again, old buddy. I'm...",good hear voice old buddy happy hear jessica b...,hello
364184,123318,123318,zufferli-j\sent_items\95#1,Trade with John Lavorato,This is a trade with OIL-SPEC-HEDGE-NG (John L...,trade oilspechedgeng john lavorato book john z...,trade john lavorato


In [ ]:
df = df.merge(df1, left_on = 'document', right_on = 'email_id', how = 'left')[['email_id','clean_body']]

In [ ]:
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe("merge_entities")

def get_ner(text):
    doc = nlp(text)
    # for ent in doc.ents:
    #   if ent.label_ == 'PERSON':
    #     s = ent.start_char
    #     e = ent.end_char
    #     text = text[:s] + 'PERSON' + text[e:]
    # return text

    return " ".join(['person' if t.ent_type_ and t.ent_type_ == 'PERSON' else t.text for t in doc])

#

In [ ]:
df['email'] = df['clean_body'].apply(get_ner)

In [ ]:
df = df[['email_id','email']]

In [ ]:
df.to_csv(data_path+"mask_name.csv")

## complete data

In [7]:
df1 = df1[['email_id','clean_subject','clean_body']]
df1

,email_id,clean_subject,clean_body
0,allen-p\all_documents\1#1,december 2000 bear stearns prediction telecom ...,today daily update find free report america on...
1,allen-p\all_documents\10#1,bloomberg power lines report,today copy bloomberg power lines adobe acrobat...
2,allen-p\all_documents\100#1,consolidated position issues list,initial set meeting trader regarding consolida...
3,allen-p\all_documents\100#3,consolidated position issues list,NaN
4,allen-p\all_documents\104#2,var reporting resources meeting,NaN
...,...,...,...
364181,zufferli-j\sent_items\85#1,generation maintenance planning related issues...,representative organization invited attend mee...
364182,zufferli-j\sent_items\86#3,hello,going great even weather record warm think hap...
364183,zufferli-j\sent_items\93#1,hello,good hear voice old buddy happy hear jessica b...
364184,zufferli-j\sent_items\95#1,trade john lavorato,trade oilspechedgeng john lavorato book john z...


In [15]:
# drop empty subject
# df1[~((df1['clean_subject']==' ' ) | (df1['clean_subject'].isna()))]
df1 = df1.dropna()

In [ ]:
df1['email_mask_name'] = df1['clean_body'].apply(get_ner)

In [ ]:
df1['subject_mask_name'] = df1['clean_subject'].apply(get_ner)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df1 = df1[['email_id','subject_mask_name','email_mask_name']].rename(columns = {'email_mask_name':'email', 'subject_mask_name':'subject'})

In [ ]:
df1.to_csv(data_path+"emails_mask_name.csv")